In [6]:
from binance.client import Client as bnb_client
import pandas as pd

client = bnb_client(tld='US')
def get_crypto_data(symbol):
    # Kline (the information we are working with): open price, high price, low price, close price, volume in the four hour period
    klines = client.get_historical_klines(symbol, '4h', '2020-01-01', '2024-12-31')
    
    df = pd.DataFrame(klines)
    df[0] = pd.to_datetime(df[0], unit='ms')
    df = df.set_index(0)[[4]]  # Just close price
    df.columns = [symbol]
    df[symbol] = df[symbol].astype(float)
    
    return df

btc_data = get_crypto_data('BTCUSDT')
print(btc_data.head())
print(f"Shape: {btc_data.shape}")

                     BTCUSDT
0                           
2020-01-01 00:00:00  7230.71
2020-01-01 04:00:00  7205.50
2020-01-01 08:00:00  7195.80
2020-01-01 12:00:00  7233.02
2020-01-01 16:00:00  7223.72
Shape: (10951, 1)


In [7]:
symbols = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT']
dfs = []


for symbol in symbols:
    df = get_crypto_data(symbol)
    dfs.append(df)


# combining into one data frame

prices = pd.concat(dfs, axis=1)
print(prices.head())

                     BTCUSDT  ETHUSDT  ADAUSDT
0                                             
2020-01-01 00:00:00  7230.71   130.18  0.03308
2020-01-01 04:00:00  7205.50   130.52  0.03320
2020-01-01 08:00:00  7195.80   130.84  0.03321
2020-01-01 12:00:00  7233.02   131.84  0.03357
2020-01-01 16:00:00  7223.72   131.98  0.03361
Shape: (10951, 3)


Because there is such a difference in price here, for our strategy we should use returns instead of just raw price data

In [8]:
returns = prices.pct_change().dropna()
print("Returns:")
print(returns.head())
print(f"Returns shape: {returns.shape}")
print("\nReturns stats:")
print(returns.describe())

Returns:
                      BTCUSDT   ETHUSDT   ADAUSDT
0                                                
2020-01-01 04:00:00 -0.003487  0.002612  0.003628
2020-01-01 08:00:00 -0.001346  0.002452  0.000301
2020-01-01 12:00:00  0.005172  0.007643  0.010840
2020-01-01 16:00:00 -0.001286  0.001062  0.001192
2020-01-01 20:00:00 -0.003886 -0.011062 -0.004463
Returns shape: (10950, 3)

Returns stats:
            BTCUSDT       ETHUSDT       ADAUSDT
count  10950.000000  10950.000000  10950.000000
mean       0.000326      0.000445      0.000526
std        0.013581      0.017206      0.021426
min       -0.200295     -0.210851     -0.231597
25%       -0.004749     -0.006277     -0.009006
50%        0.000331      0.000297      0.000198
75%        0.005319      0.007360      0.009403
max        0.150512      0.144593      0.157946
